In [ ]:
import math
from itertools import product

In [ ]:
sc.uninstall_package('pip')
sc.install_pypi_package("pip==22.2.2")
sc.install_pypi_package("opencv-python") #Install opencv-python 
sc.install_pypi_package("numpy")

In [ ]:
def deg2num(lat_deg, lon_deg, zoom):
    '''
        Convert latitude, longitude to (x,y) tile coordinate at given zoom.
    '''
    lat_rad = math.radians(lat_deg)
    n = 2.0**zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)

In [ ]:
def num2deg(xtile, ytile, zoom):
    '''
        Convert (x,y) to latitude, longitude tile coordinate at given zoom.
    '''
    n = 2.0**zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

In [ ]:
def tiles_url(zoom, lat1, lon1, lat2, lon2):
    ''' 
        Convert geographic bounds into a list of tile_url coordinates at given zoom.
    '''
    # convert to geographic bounding box
    minlat, minlon = min(lat1, lat2), min(lon1, lon2)
    maxlat, maxlon = max(lat1, lat2), max(lon1, lon2)

    # convert to tile-space bounding box
    xmin, ymin = deg2num(maxlat, minlon, zoom)
    xmax, ymax = deg2num(minlat, maxlon, zoom)

    # generate a list of tiles_url
    xs, ys = range(xmin, xmax + 1), range(ymin, ymax + 1)
    tiles_url = [(f"s3://elevation-tiles-prod//terrarium/{zoom}/{x}/{y}.png")
                 for (y, x) in product(ys, xs)]

    return tiles_url

In [ ]:
import numpy as np
import cv2

In [6]:
def main():
    data = tiles(7, 30, -85, 24, -79)
    input_data = spark.read.format("image").load(data)
    
    images = input_data.orderBy("image.origin").select("image.data")
    
    images = images.rdd.map(lambda img: np.reshape(img, (256,256,3)))
    images = images.map(lambda img: cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    
    # gradient 
    #gX = images.map(lambda img: cv2.Sobel(images, cv2.CV_64F, 1, 0))
    #gY = images.map(lambda img: cv2.Sobel(images, cv2.CV_64F, 0, 1))
    #magnitude = images.map(lambda img:np.sqrt((gX ** 2) + (gY ** 2)))
    
    dataColl=images.collect()
    print(dataColl)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['s3://elevation-tiles-prod//terrarium/7/33/52.png', 's3://elevation-tiles-prod//terrarium/7/34/52.png', 's3://elevation-tiles-prod//terrarium/7/35/52.png', 's3://elevation-tiles-prod//terrarium/7/33/53.png', 's3://elevation-tiles-prod//terrarium/7/34/53.png', 's3://elevation-tiles-prod//terrarium/7/35/53.png', 's3://elevation-tiles-prod//terrarium/7/33/54.png', 's3://elevation-tiles-prod//terrarium/7/34/54.png', 's3://elevation-tiles-prod//terrarium/7/35/54.png', 's3://elevation-tiles-prod//terrarium/7/33/55.png', 's3://elevation-tiles-prod//terrarium/7/34/55.png', 's3://elevation-tiles-prod//terrarium/7/35/55.png']

In [7]:
main()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|               image|
+--------------------+
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
+--------------------+